### RNN Training Notebook  

This notebook trains an AI model to predict mouse cursor movement paths. The model is built using a Recurrent Neural Network (RNN) with an LSTM layer for attention.

In [14]:
import os
import json
import torch
import numpy as np
from collections import Counter
import torch.nn as nn
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
dataset_path = os.path.join(os.path.dirname(os.getcwd()), "data", "processed", 'cleaned-data-39-steps-merged-prepared-data-2025-03-08-16:27:23.json') # this path is for the cleaned data; must be changed accordingly

print(f"{dataset_path} exists: {"Yes" if os.path.exists(dataset_path) else 'No'}") ## must be Yes

/Users/arpanbhandari/Documents/coding/bumblebee/data/processed/cleaned-data-39-steps-merged-prepared-data-2025-03-08-16:07:40.json exists: Yes


In [16]:
dataset_file = open(dataset_path, 'r')
dataset_json = json.load(dataset_file)
dataset_file.close()

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps')

In [22]:
input_data = np.array(dataset_json['input'], dtype=np.float64)
output_data = np.array(dataset_json['output'], dtype=np.float64)

# print(Counter([len(x) for x in dataset_json['output']]))

# # Check for non-integer elements in the output data
# non_int_examples = [x for x in dataset_json['output'] if any(not (isinstance(point, int) or isinstance(point, float)) for point in x)]
# print(f"Found {len(non_int_examples)} examples with non-integer values")
# if non_int_examples:
#     print("First few non-integer examples:")
#     for i, example in enumerate(non_int_examples[:10]):
#         print(f"Example {i}: {example[:5]}...")

In [23]:
input_data = torch.tensor(input_data, dtype=torch.float32).to(device)
output_data = torch.tensor(output_data, dtype=torch.float32).to(device)